***Load Packages***

In [1]:
import re
import pandas as pd
from IPython.display import display_html

from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.comparison_library import exact_match, levenshtein_at_thresholds
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_level_library as cll


C:\Users\Kameron\AppData\Local\Temp\ipykernel_4232\3623306441.py:6: SplinkDeprecated: Importing directly from `splink.duckdb.duckdb_comparison_library` is deprecated and will be removed in Splink v4. Please import from `splink.duckdb.comparison_library` going forward.
  from splink.duckdb.duckdb_comparison_library import (exact_match, levenshtein_at_thresholds)


__Load Data and prevent panda from placing "NaN" into blank cells__

In [4]:
# Load the CSV files with dtype specified as str to ensure all data is read as strings
iss_c = pd.read_csv(r'C:\Users\Kameron\Documents\ESG Thesis\Data\Control Variables\Board Level\iss.csv', dtype=str, low_memory=False)
comp_c = pd.read_csv(r'C:\Users\Kameron\Documents\ESG Thesis\Data\Control Variables\Board Level\comp.csv', dtype=str, low_memory=False)

# Add a unique_id column starting at 1
iss_c['unique_id'] = range(1, len(iss_c) + 1)
comp_c['unique_id'] = range(1, len(comp_c) + 1)

iss_m = iss_c
comp_m = comp_c

# Replace NaN values with an empty string in both DataFrames
iss_c.fillna("", inplace=True)
comp_c.fillna("", inplace=True)


__This Section Regaurds Institutional Shareholder Solutions (ISS) Data__

*Renaming Columns*

In [5]:
iss_c.rename(columns={'first_name': 'first', 'last_name': 'last', 'fullname': 'full'}, inplace=True)

__Establishing Functions__

*The first function is called "standardize_names". This functions cleans the first name column by ensuring the first name comes first and any middle initals or names come second. This adjustment standardizes all the data so that another function (which will be mentioned next) can delineate the first names from the middles names, thus providing data handlers with two seperate columns called "first" and "middle"*

In [6]:
import re
import pandas as pd

# List of common titles with regular expression format for easier matching
titles = [r"JR\.", r"SR\.", r"II", r"III", r"IV", r"M\.D\.", r"PH\.D\."]

# Function to process name strings and separate titles from the main name
def process_names(name):
    """
    This function takes a single name string as input, searches for a predefined list of titles at the end of the name,
    removes the title from the name, and returns both the cleaned name and the title separately.
    """
    # Compile a regular expression pattern to find titles, which are optionally preceded by a comma or space
    pattern = re.compile(r',?\s*(' + '|'.join(titles) + r')$')
    match = pattern.search(name)
    if match:
        # If a title is found, remove it from the name
        name = pattern.sub('', name)
        # Return the cleaned name and the found title, stripped of dots for standardization
        return name.strip(), match.group(1).replace('.', '')
    # If no title is found, return the original name and None for the title
    return name, ""

# Function to standardize name strings
def standardize_names(names):
    """
    This function takes a list of name strings, removes any content in parentheses or double quotes,
    normalizes whitespace, converts all to lowercase, and returns the standardized names.
    """
    standardized = []
    for name in names:
        # Remove content in parentheses and double quotes
        name = re.sub(r'\(.*?\)', '', name)
        name = re.sub(r'\".*?\"', '', name)
        # Replace multiple spaces with a single space, trim, and convert to lower case
        name = re.sub(r'\s+', ' ', name).strip().lower()
        standardized.append(name)
    return standardized

# DataFrame operations assuming 'iss_c' is a predefined DataFrame with 'full' column
# Applying 'process_names' to 'full' column and splitting results into new 'full' and 'title' columns
iss_c['full'], iss_c['title'] = zip(*iss_c['full'].apply(process_names))
# Applying 'standardize_names' to cleaned 'full' column
iss_c['full'] = standardize_names(iss_c['full'])

# Function to split a name into first, middle, and last components
def split_name(name):
    """
    This function splits a single cleaned name into first, middle, and last name components based on whitespace and initials.
    """
    # Split the name by spaces after trimming excess whitespace
    parts = re.split(r'\s+', name.strip())
    # Initialize variables for first, middle, and last names
    first = ''
    middle = ''
    last = ''
    
    # Determine how to assign name parts based on the number of components
    if len(parts) == 1:
        # Only one part, assume it's the first name
        first = parts[0]
    elif len(parts) == 2:
        # Two parts, assign first and last names
        first, last = parts
    else:
        # More than two parts, check for initials and assign accordingly
        initial_indices = [i for i, part in enumerate(parts) if re.match(r'^[a-zA-Z]\.$', part)]
        if initial_indices:
            if initial_indices[0] == 0 and len(initial_indices) == 1:
                middle = parts[0]  # One initial, assume it's a middle name
                first = parts[1]
                last = ' '.join(parts[2:])
            else:
                first = parts[0]
                middle = ' '.join(parts[1:-1])
                last = parts[-1]
        else:
            first = parts[0]
            middle = ' '.join(parts[1:-1])
            last = parts[-1]
    
    return first, middle, last

# Applying 'split_name' to the standardized 'full' column in 'iss_c', splitting results into 'first', 'middle', 'last' columns
iss_c[['first', 'middle', 'last']] = iss_c['full'].apply(lambda x: pd.Series(split_name(x)))


In [7]:
# Specifying the order of the columns
column_order = ['title', 'first', 'middle', 'last', 'full']  # desired first columns

# Add the rest of the columns dynamically
column_order.extend([col for col in iss_c.columns if col not in column_order])

# Reindex the DataFrame according to the new column order
iss_c = iss_c[column_order]

*Clean Names*

In [8]:
# List of columns to clean
columns_to_clean = ['title', 'first', 'middle', 'last', 'full']

# Removing non-alphabetical characters and spaces
for column in columns_to_clean:
    iss_c[column] = iss_c[column].str.replace('[^a-zA-Z]', '', regex=True)

*convert the 9 digit cusip ids to 8 digit cusips*

In [9]:
iss_c['cusip'] = iss_c['cusip'].str[:-1]

*Concatenate the year and cusip column. This will be important for using a blocking function with SPlink later.*

In [10]:
iss_c['cy'] = iss_c['cusip'].str.zfill(8) + iss_c['year'].astype(str)

# Prepping Capital IQ Compustat Data

__Clean data for standardization__


*Make column headers and data lowercase*

In [11]:
comp_c.columns = comp_c.columns.str.lower()
comp_c = comp_c.apply(lambda col: col.apply(lambda x: x.lower() if isinstance(x, str) else x))

*Rename Director Column*

In [12]:
comp_c = comp_c.rename(columns={'dirname': 'full'})


*Add a unique_id column (important for SPlink later)*

In [13]:
comp_c.loc[:, 'unique_id'] = range(1, len(comp_c) + 1)

*Concatenate cusip and year*

In [14]:
comp_c['cy'] = comp_c['cusip'] + comp_c['year']

**Split the 'full' column into three new columns and clean them. Then clean the full column.**

In [15]:
# Titles to look for
titles = [r"jr\.", r"sr\.", r"ii", r"iii", r"iv", r"md\.", r"phd\."]

# Regex to capture titles from the list, preceded by a comma
title_regex = r',\s*(' + '|'.join(titles) + r')'

# Function to extract titles and clean the 'full' name
def extract_and_clean(row):
    import re
    # Find all titles in the 'full' column
    found_titles = re.findall(title_regex, row['full'])
    
    # Prioritize family titles over academic ones
    family_titles = [t for t in found_titles if t in ['JR.', 'SR.', 'II', 'III', 'IV']]
    if family_titles:
        # Select the first family title found (since they are prioritized)
        selected_title = family_titles[0]
    elif found_titles:
        # If no family titles, select the first academic title found
        selected_title = found_titles[0]
    else:
        # If no titles are found, return None
        selected_title = ""
    
    # Remove anything after the first comma (including the comma itself)
    cleaned_name = re.split(r',', row['full'])[0]
    
    return pd.Series([cleaned_name, selected_title])

# Apply the function and update the DataFrame
comp_c[['full', 'title']] = comp_c.apply(extract_and_clean, axis=1)


In [16]:
# Function to split names into first, middle, and last
def split_name(name):
    # Remove extra spaces and split the name
    parts = re.split(r'\s+', name.strip())
    first = ''
    middle = ''
    last = ''
    
    # Analyze the parts to assign to first, middle, last
    if len(parts) == 1:
        first = parts[0]  # Only one part, assume first name
    elif len(parts) == 2:
        first, last = parts
    else:
        # Check for initials in parts
        initial_indices = [i for i, part in enumerate(parts) if re.match(r'^[a-zA-Z]\.$', part)]
        if initial_indices:
            # If the first part is an initial and directly precedes the first name
            if initial_indices[0] == 0 and len(initial_indices) == 1:
                middle = parts[0]  # Just one initial as middle name
                first = parts[1]
                last = ' '.join(parts[2:])
            else:
                first = parts[0]
                middle = ' '.join(parts[1:-1])
                last = parts[-1]
        else:
            first = parts[0]
            middle = ' '.join(parts[1:-1])
            last = parts[-1]
    
    return first, middle, last


# Apply the function to each name in the 'dirname' column
comp_c[['first', 'middle', 'last']] = comp_c['full'].apply(lambda x: pd.Series(split_name(x)))

In [17]:
# Specifying the order of the columns
column_order = ['title','first', 'middle', 'last', 'full']  # desired first columns

# Add the rest of the columns dynamically
column_order.extend([col for col in comp_c.columns if col not in column_order])

# Reindex the DataFrame according to the new column order
comp_c = comp_c[column_order]

# List of columns to clean
columns_to_clean = ['first', 'middle', 'last', 'full', 'title']

# Removing non-alphabetical characters (and spaces)
for column in columns_to_clean:
    comp_c[column] = comp_c[column].str.replace('[^a-zA-Z]', '', regex=True)

In [18]:
comp_c 

,title,first,middle,last,full,gvkey,dirnbr,cash_fees,stock_awards,option_awards,...,naicsdesc,inddesc,spcode,ticker,sub_tele,naics,spindex,sic,unique_id,cy
0,,marc,jay,walfish,marcjaywalfish,001004,1,80,71.64,0,...,transportation equipment and supplies (except ...,aerospace & defense,sm,air,630,423860,2010,5080,1,000361102010
1,,ronald,b,woodard,ronaldbwoodard,001004,2,82.5,71.64,0,...,transportation equipment and supplies (except ...,aerospace & defense,sm,air,630,423860,2010,5080,2,000361102010
2,,patrick,j,kelly,patrickjkelly,001004,3,72.5,71.64,0,...,transportation equipment and supplies (except ...,aerospace & defense,sm,air,630,423860,2010,5080,3,000361102010
3,,james,e,goodwin,jamesegoodwin,001004,4,85,71.64,0,...,transportation equipment and supplies (except ...,aerospace & defense,sm,air,630,423860,2010,5080,4,000361102010
4,,ronald,r,fogleman,ronaldrfogleman,001004,5,78.75,71.64,0,...,transportation equipment and supplies (except ...,aerospace & defense,sm,air,630,423860,2010,5080,5,000361102010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246807,,joseph,,alvarado,josephalvarado,328795,5,130,130.046,0,...,fabricated structural metal manufacturing,construction & engineering,sm,aca,972,332312,2010,3440,246808,039653102023
246808,,kimberly,s,lubel,kimberlyslubel,328795,6,110,130.046,0,...,fabricated structural metal manufacturing,construction & engineering,sm,aca,972,332312,2010,3440,246809,039653102023
246809,,julie,a,piggott,julieapiggott,328795,7,110,130.046,0,...,fabricated structural metal manufacturing,construction & engineering,sm,aca,972,332312,2010,3440,246810,039653102023
246810,,jeffrey,a,craig,jeffreyacraig,328795,8,130,130.046,0,...,fabricated structural metal manufacturing,construction & engineering,sm,aca,972,332312,2010,3440,246811,039653102023


In [19]:
# Calculate the count of unique CUSIPs per year for both dataframes
unique_cusip_per_year_iss = iss_c.groupby('year')['cusip'].nunique()
unique_cusip_per_year_comp = comp_c.groupby('year')['cusip'].nunique()

# Create a DataFrame to compare the counts from both datasets
comparison_df = pd.DataFrame({
    'Unique CUSIPs in ISS': unique_cusip_per_year_iss,
    'Unique CUSIPs in COMP': unique_cusip_per_year_comp
})

# Calculate the absolute difference in unique CUSIPs per year between the two datasets
comparison_df['Difference'] = comparison_df['Unique CUSIPs in ISS'].sub(comparison_df['Unique CUSIPs in COMP']).abs()

# Optionally, calculate the percentage match
comparison_df['Match Percentage'] = 100 * (1 - (comparison_df['Difference'] / comparison_df[['Unique CUSIPs in ISS', 'Unique CUSIPs in COMP']].max(axis=1)))

# Display the comparison DataFrame
print(comparison_df)

      Unique CUSIPs in ISS  Unique CUSIPs in COMP  Difference  \
year                                                            
2010                  1481                   2229         748   
2011                  1470                   2192         722   
2012                  1497                   2173         676   
2013                  1515                   2169         654   
2014                  1499                   2163         664   
2015                  1505                   2106         601   
2016                  1508                   2042         534   
2017                  1515                   1979         464   
2018                  1506                   1947         441   
2019                  1526                   1888         362   
2020                  1519                   1859         340   
2021                  1491                   1819         328   
2022                  1515                   1782         267   
2023                  154

Compare the data sets

In [20]:

# Filter both DataFrames for rows where 'cy' equals '000360202010'
comp_c_filtered = comp_c[comp_c['cy'] == '000360202011']
iss_c_filtered = iss_c[iss_c['cy'] == '000360202011']

# Since we're now directly targeting specific entries, sorting may not be necessary, but we'll maintain structure
comp_c_sorted = comp_c_filtered[['unique_id', 'cusip', 'year', 'first', 'middle', 'last', 'full', 'cy']].head(5)
iss_c_sorted = iss_c_filtered[['unique_id', 'cusip', 'year', 'first', 'middle', 'last', 'full', 'cy']].head(5)

# Convert dataframes to HTML
comp_c_html = comp_c_sorted.to_html()
iss_c_html = iss_c_sorted.to_html()
space = '&nbsp;'*10  # Adjust space as needed

# Combine the HTML strings with space in between
html_string = f'<div style="float: left;">{comp_c_html}</div>{space}<div style="float: left;">{iss_c_html}</div>'

# Display the HTML string in the notebook
display_html(html_string, raw=True)


,unique_id,cusip,year,first,middle,last,full,cy
119763,119764,00036020,2011,john,b,johnson,johnbjohnson,000360202011
119764,119765,00036020,2011,arthur,h,mcelroy,arthurhmcelroy,000360202011
119765,119766,00036020,2011,paul,kenneth,lackey,paulkennethlackey,000360202011
119766,119767,00036020,2011,jack,e,short,jackeshort,000360202011
119767,119768,00036020,2011,joseph,e,cappy,josephecappy,000360202011
,unique_id,cusip,year,first,middle,last,full,cy
629,630,00036020,2011,joseph,e,cappy,josephecappy,000360202011
630,631,00036020,2011,norman,h,asbjornson,normanhasbjornson,000360202011
631,632,00036020,2011,john,b,johnson,johnbjohnson,000360202011
632,633,00036020,2011,jack,e,short,jackeshort,000360202011


# Merging Compustat and ISS (Former risk metrics)

In [21]:
# Selecting specific columns from comp_c to create df_r
df_r = comp_c[['unique_id', 'cusip', 'year', 'cy', 'first', 'middle', 'last', 'full']]

# Selecting specific columns from iss_c to create df_l
df_l = iss_c[['unique_id', 'cusip', 'year', 'cy', 'first', 'middle', 'last', 'full']]



h*Double Check Data Types Mefore Merge*

In [22]:
# Get the data types of the columns
df_r_dtypes = df_r.dtypes
df_l_dtypes = df_l.dtypes

# Print the data types
print("Data types of columns in df_r:")
print(df_r_dtypes)
print("\nData types of columns in df_l:")
print(df_l_dtypes)

Data types of columns in df_r:
unique_id     int64
cusip        object
year         object
cy           object
first        object
middle       object
last         object
full         object
dtype: object

Data types of columns in df_l:
unique_id     int64
cusip        object
year         object
cy           object
first        object
middle       object
last         object
full         object
dtype: object


*SPlink settings*

In [23]:

settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        block_on("cy",salting_partitions=2)
    ],
    "comparisons": [
        ctl.name_comparison("first"),
        ctl.name_comparison("full"),
        ctl.name_comparison("last"),
        ctl.name_comparison("middle"),
        cl.exact_match("year"),
        cl.exact_match("cusip"),
    ],       
}

linker = DuckDBLinker([df_l, df_r], settings,connection=":temporary:", input_table_aliases=["comp", "iss"])

*Check data quality*

linker.completeness_chart()

*Deterministic rules are crucial in the linking process, defining which comparisons are accepted or rejected. I use the Jaro-Winkler similarity method, which prioritizes the match of the initial characters in two strings and de-emphasizes later discrepancies. This approach is valuable when dealing with slight variations in dataset spelling. For example, "Lee Wan Hu" and "Lee W. H." would score high, indicating a strong match. However, "Kyle Moster" vs. "Lyle Mester" would score low due to different initial characters, leading my deterministic rules to exclude such comparisons from progressing to more rigorous matching processes. This ensures efficiency by focusing on the most likely matches.*

In [24]:
deterministic_rules = [
    
    "jaro_winkler_similarity(r.full, l.full) >= .6 and r.cy = l.cy and jaro_winkler_similarity(r.first, l.first) >= .6 and jaro_winkler_similarity(r.last, l.last) >= .6",

]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=.9)

Probability two random records match is estimated to be  2.38e-06.
This means that amongst all possible pairwise record comparisons, one in 420,519.23 are expected to match.  With 49,244,670,676 total possible comparisons, we expect a total of around 117,104.44 matching pairs


In [25]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first (no m values are trained).
    - full (no m values are trained).
    - last (no m values are trained).
    - middle (no m values are trained).
    - year (no m values are trained).
    - cusip (no m values are trained).


In [26]:
session_cusip_year = linker.estimate_parameters_using_expectation_maximisation(block_on("cy"))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."cy" = r."cy"

Parameter estimates will be made for the following comparison(s):
    - first
    - full
    - last
    - middle
    - year
    - cusip

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Level All other comparisons on comparison year not observed in dataset, unable to train m value

Level All other comparisons on comparison cusip not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.531 in the m_probability of full, level `Exact match full`
Iteration 2: Largest change in params was -0.12 in the m_probability of last, level `Exact match last`
Iteration 3: Largest change in params was 0.806 in the m_probability of first, level `All other comparisons`
Iteration 4: Largest change in params was 0.282 in probability_two_random_records_match
Iteration 5: Largest change in params was 1.15e-05 in probability_two_random_records_match

EM converged after 5 iterations
m probability not trained for year - All other comparisons (comparison vector value: 0). This usually means the comparison level was never observed in the training data.
m probability not trained for cusip - All other comparisons (comparison vector value: 0). This usually mea

In [27]:
results = linker.predict(threshold_match_probability=0.8)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'year':
    m values not fully trained
Comparison: 'cusip':
    m values not fully trained


In [51]:
results.as_pandas_dataframe()

df_r=comp_

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_l,first_r,gamma_first,full_l,...,middle_r,gamma_middle,year_l,year_r,gamma_year,cusip_l,cusip_r,gamma_cusip,cy_l,cy_r
0,2.389267,0.839716,comp,iss,46674,116612,uwe,uwe,4,uwerohrhoff,...,f,3,2017,2017,1,14880610,14880610,1,148806102017,148806102017
1,2.396077,0.840350,comp,iss,175478,74313,lars,lars,4,larssoerensen,...,rebien,0,2022,2022,1,88355610,88355610,1,883556102022,883556102022
2,2.396077,0.840350,comp,iss,98288,81663,william,william,4,williampesce,...,edward,0,2019,2019,1,96822320,96822320,1,968223202019,968223202019
3,2.396077,0.840350,comp,iss,98299,81674,william,william,4,williampesce,...,edward,0,2020,2020,1,96822320,96822320,1,968223202020,968223202020
4,2.396077,0.840350,comp,iss,142060,91814,ralf,ralf,4,ralfrboer,...,reinhard,0,2012,2012,1,72913210,72913210,1,729132102012,729132102012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101557,18.971904,0.999998,comp,iss,28742,245837,theodore,theodore,4,theodoremarmstrong,...,m,4,2014,2014,1,12680430,12680430,1,126804302014,126804302014
101558,18.971904,0.999998,comp,iss,2086,246803,antonio,antonio,4,antoniocarrillo,...,,4,2022,2022,1,03965310,03965310,1,039653102022,039653102022
101559,18.971904,0.999998,comp,iss,129514,245306,lesley,lesley,4,lesleyhhowe,...,h,4,2013,2013,1,67070410,67070410,1,670704102013,670704102013
101560,18.971904,0.999998,comp,iss,28713,245818,theodore,theodore,4,theodoremarmstrong,...,m,4,2011,2011,1,12680430,12680430,1,126804302011,126804302011


In [58]:
results_df = results.as_pandas_dataframe()

# Perform the left join
m1 = results_df.merge(comp_c, left_on='unique_id_r', right_on='unique_id', how='left')
m2 = m1.merge(iss_c, left_on='unique_id_l', right_on='unique_id', how='left')

In [64]:
m2.to_csv(r'C:\Users\Kameron\Documents\ESG Thesis\Data\Control Variables\Board Level\iss_comp_merge.csv')